# Visualizing Traffic Accidents Around LA with IBM Cloud SQL Query and PixieDust

This notebook contains a demonstration on how to use IBM Cloud SQL Query in a Jupyter Notebook, and renders the data returned from SQL Query with PixieDust. For more information, see ...

To use this notebook, you'll need:

- IBM Cloud Object Storage and create a bucket for some data
- IBM Cloud SQL Query

You'll first need to download the [traffic accident CSV file](https://catalog.data.gov/dataset/traffic-collision-data-from-2010-to-present/resource/643d0e98-5f40-4db3-8427-02641dd05fd9?inner_span=True) and upload it to your Cloud Object Storage (COS) bucket. 
 
Now, run the next cell to download the latest versions of ibmcloudsql and PixieDust.

In [ ]:
!pip -q install ibmcloudsql
!pip install --upgrade pixiedust

Import both into the notebook:

In [ ]:
import ibmcloudsql
import pixiedust

In order to keep your credentials safe, import `getpass`, which will store your IBM Cloud API Key so it's not visible to people viewing the notebook. You can enter any prompt you'd like as a string. Once you run the cell, a box will appear. Paste your IBM Cloud API Key into the box and hit return. All you have to do is use the variable `cloud_api_key` to use it.

In [ ]:
import getpass
cloud_api_key = getpass.getpass('Enter your IBM Cloud API Key')

The next variables contain the SQL Query CRN and the endpoint of the COS bucket generated when provisioning SQL Query. For the CRN, go to your SQL Query service page - you'll be on the **Manage** tab. Under, _REST API_ there is a button **Instance CRN**. Click that to copy the CRN for the service. Then add that to in the quotes below.

For the COS endpoint that will contain the SQL Query results, you can use any bucket you'd like; however, SQL Query generates one for you dso we'll use that one. To the endpoint, we've added the suffix `/accidents`. This will be the prefix of all the results that will be CSV files generated by SQL Query.

In [ ]:
sql_crn = 'crn%3Av1%3Abluemix%3Apublic%3Asql-query%3Aus-south%3Aa%2Fd9333eaa1345bf35cbec45eaf7048d6b%3Af5ab81ab-4152-4cbf-8376-10204ce5a5a8%3A%3A' 
sql_cos_endpoint = 'cos://s3.us-south.objectstorage.softlayer.net/sql-f5ab81ab-4152-4cbf-8376-10204ce5a5a8/accicents'

Run `ibmcloudsql.SQLQuery` with the API Key, CRN, and COS endpoint. You'll then have access to the `run_sql` method to run your SQL queries on the data.  Here `cos://region/bucket_name/data_file` is the table unique resource identifier (equivalent to a table in SQL) that can be a CSV, Parquet, or JSON file. 

You can also join files together with with an SQL join, or you can use a wildcard on files names like `example-*` to include several files starting with that name.

The following query gets the time, area, age (between 20-35), victim sex, and location of accidents between 5pm and 8pm.

In [ ]:
sqlClient = ibmcloudsql.SQLQuery(cloud_api_key, sql_crn, sql_cos_endpoint)

data_source = "cos://us-geo/<your_COS_bucket>/Traffic_Collision_Data_from_2010_to_Present.csv"

query = """
SELECT 
    `Time Occurred` AS time, 
    `Area Name` AS area, 
    `Victim Age` AS age, 
    `Victim Sex` AS sex, 
    `Location` AS location 
FROM  {}
WHERE 
    `Time Occurred` >= 1700 AND `Time Occurred` <= 2000 AND 
    `Victim Age` >= 20 AND `Victim Age` <= 35
""".format(data_source)

traffic_collisions = sqlClient.run_sql(query)

After the query runs, we can look at a sample of the results.

In [ ]:
traffic_collisions.head()

SQL Query can also handle more advanced queries like CTEs (common table expressions). In the following example, the CTE formats the _location_ column of the previous query and divides the coordinates into separate latitude and longitude columns.

In [ ]:
sqlClient = ibmcloudsql.SQLQuery(cloud_api_key, sql_crn, sql_cos_endpoint)

traffic = sqlClient.run_sql("""
WITH location AS ( 
    SELECT 
        id, 
        cast(split(coordinates, ',')[0] as float) as latitude, 
        cast(split(coordinates, ',')[1] as float) as longitude 
    FROM (SELECT 
            'Dr Number` as id, 
            regexp_replace(`Location`, '[()]', '') as coordinates 
        FROM cos://us-geo/<your_COS_bucket>/Traffic_Collision_Data_from_2010_to_Present.csv  
    ) 
) 
SELECT  
    d.`Dr Number` as id, 
	d.`Date Occurred` as date, 
    d.`Time Occurred` AS time, 
    d.`Area Name` AS area, 
    d.`Victim Age` AS age, 
    d.`Victim Sex` AS sex, 
    l.latitude, 
    l.longitude 
FROM cos://us-geo/<your_COS_bucket>/Traffic_Collision_Data_from_2010_to_Present.csv AS d 
    JOIN 
    location AS l 
    ON l.id = d.`Dr Number` 
WHERE 
    d.`Time Occurred` >= 1700 AND 
    d.`Time Occurred` <= 2000 AND 
    d.`Victim Age` >= 20 AND 
    d.`Victim Age` <= 35 AND 
    l.latitude != 0.0000 AND 
    l.latitude != 0.0000
""")


In [ ]:
traffic.head()

Using PixieDust, we can view the locations of these traffic accidents on a map. Select _Map_ as the chart type. Then, drag _latitude_ and _longitude_ to the _Keys_ box and _id_, _age_, _sex_, and _date_ to the _Values_ box.

In [ ]:
display(traffic)